In [14]:
%load_ext autoreload
%autoreload 2

import os
from jax import config
config.update("jax_enable_x64", True)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
from encoding_information.gpu_utils import limit_gpu_memory_growth
limit_gpu_memory_growth()

import tensorflow_datasets as tfds  # TFDS for MNIST
import tensorflow as tf             # TensorFlow operations

# from image_distribution_models import PixelCNN

from cleanplots import *
import jax.numpy as np
from jax.scipy.special import logsumexp
import numpy as onp
from encoding_information.image_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
#MNIST
(patches, _), (_ , _) = tf.keras.datasets.mnist.load_data()
patches = (patches.astype(np.float32) / 255.) * 300 + 25 

patches = add_noise(patches)


In [45]:
from encoding_information.information_estimation import estimate_mutual_information

mi_gaussian, stationary_gaussian  = estimate_mutual_information(patches, eigenvalue_floor=1e0, entropy_model='gaussian', max_epochs=10, verbose=True, return_entropy_model=True)
mi_pixelcnn, pixel_cnn = estimate_mutual_information(patches, entropy_model='pixelcnn', max_epochs=20, verbose=True, return_entropy_model=True)

Computing loss: 15it [00:01, 11.33it/s]


Initial validation NLL: 11.45


Epoch 1: 100%|██████████| 100/100 [00:08<00:00, 11.99it/s]
Computing loss: 15it [00:00, 82.60it/s]


Epoch 1: validation NLL: 3.49


Epoch 2: 100%|██████████| 100/100 [00:03<00:00, 30.64it/s]
Computing loss: 15it [00:00, 92.15it/s]


...

Epoch 19: validation NLL: 3.43


Epoch 20: 100%|██████████| 100/100 [00:02<00:00, 34.12it/s]
Computing loss: 15it [00:00, 92.53it/s]


Epoch 20: validation NLL: 3.43


Computing loss:   0%|          | 0/32 [00:00<?, ?it/s]2023-10-22 16:11:57.409963: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:808] None of the algorithms provided by cuDNN heuristics worked; trying fallback algorithms.
2023-10-22 16:11:57.409987: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:811] Conv: (f64[188,128,28,28]{3,2,1,0}, u8[0]{0}) custom-call(f64[188,64,28,28]{3,2,1,0}, f64[128,64,3,3]{3,2,1,0}, f64[128]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}
Computing loss:  50%|█████     | 16/32 [00:29<00:07,  2.03it/s]2023-10-22 16:12:26.688079: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:808] None of the algorithms provided by cuDNN heuristics worked; trying fallback algorithms.
2023-10-22 16:12:26.688108: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:811] Conv

Estimated H(Y|X) = 3.215 differential entropy/pixel
Estimated H(Y) (upper bound) = 3.436 differential entropy/pixel
Estimated I(Y;X) = 0.319 bits/pixel


In [46]:
print('Gaussian MI: ', mi_gaussian)
print('PixelCNN MI: ', mi_pixelcnn)

Gaussian MI:  2.790793270544653
PixelCNN MI:  0.31865013
